In [1]:
import random, re, json, pathlib, logging, time, argparse, pprint 
import pandas as pd
import numpy as np
import circlify as crcf
import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import importlib.resources as rsrc

#import postprocess.postprocess as pspr


In [ ]:
target_df = pd.read_pickle("rrp_target_df.pkl")

goal_df = pd.read_pickle("rrp_goal_df.pkl")

In [ ]:
goal_df.head()

In [ ]:

target_df['Target'] = pspr.stringify_id(target_df['Target'])

results_dict = {}

with rsrc.path("postprocess", "goal_target_list.xlsx") as res_path:
    sdg_df = pd.read_excel(res_path)

results_dict['target_dat'] = pspr.aggregate_to_targets(target_df, sdg_df)

results_dict['goal_dat'] = pspr.aggregate_to_goals(goal_df, sdg_df)

results_dict['goals_grouped_by_document'] = pspr.group_byNAme_and_get_goaloverview(results_dict['target_dat'], results_dict['goal_dat'], sdg_df)

In [ ]:
jrc = pd.read_excel('output/JRC_/output/6-results/results_.xlsx', sheet_name='goals_grouped_by_documents')
D_units = pd.read_excel('output/JRC_/output/6-results/Projects overview_DirD.xlsx',usecols=['PP','PRJ','Unit'])
jrc.head()
#jrc.drop(columns='Policy', inplace=True)
jrc_folder = jrc['Policy'].str.split('/',expand=True)
jrc_folder['Policy'] = jrc_folder[[0,1]].agg(lambda x: f'{x[0]}/{x[1]}', axis=1)
jrc_folder['PP']=jrc_folder[1].apply(lambda doc: int(re.search(r'(?<=PP_)\d{5}(?<!_)',doc).group(0)))
jrc_folder['PRJ']=jrc_folder[1].apply(lambda doc: int(re.search(r'(?<=_PRJ_)\d{5}(?<!_\d{6}.docx)',doc).group(0)))
jrc_folder = jrc_folder.join(D_units.set_index('PRJ'), on='PRJ', how='inner',lsuffix='_re', rsuffix='_jpb')
jrc_folder = jrc_folder.drop_duplicates().reset_index()
jrc = jrc.merge(jrc_folder, on='Policy', how='right')
jrc = jrc.groupby([0,'Unit','goal_id'])
DirD = jrc['Count'].sum().loc['DirD']
#DirD.head(50)
dird_dict=DirD.to_dict()
DirD.to_excel('DirD_gl.xlsx')
#print(DirD.to_list())#.pivot('Subject_id', 'Subject', 'Score').to_dict('index')
# print(jrc.columns.values.tolist())
# jrc=jrc.drop(columns=['Unnamed: 1', 'index_x', 'index_y','PP_re'])
#jrc.pivot([0,'Unit','PRJ'], 'goal_id', 'Count').to_dict('index')

# jrc_bubblechart={'name': 'sdgs', 'children':[]}

# units = list(set([unit for unit, id in dird_dict.keys()]))

# print(units)

# #{'name': 'sdgs'}
# total_value = 0
# unit=0
# for key, value in dird_dict.items():
#     temp_dict={'name': f'SDG {key[1]}', 'size':value}
#     #total_value+=value
#     jrc_bubblechart['children'].append({'name':key[0],'size':total_value,'children':[temp_dict]})
#     # jrc_bubblechart['name']=key[0]
#     # jrc_bubblechart['children'

# print(jrc_bubblechart)

In [ ]:
jrc.head()

In [ ]:
#create grouping variable by path
folders=results_dict['goals_grouped_by_document']['Policy'].str.split('/',expand=True)
folders=folders.join(results_dict['goals_grouped_by_document'])
#folders.head()
folders=folders.groupby([0, 'Goal'])

In [ ]:
#folders.head()
folders.get_group('RRP_IT')

In [ ]:
# results_dict['goals_grouped_by_document']=results_dict['goals_grouped_by_document'].drop(columns = results_dict['goals_grouped_by_document'].columns[[0]], inplace=True)

In [ ]:
results_dict['goals_grouped_by_document']=results_dict['goals_grouped_by_document'].rename_axis(['names',0])

In [ ]:
results_dict['goals_grouped_by_document'].groupby("names").groups
#results_dict['goals_grouped_by_document'].groups 

In [ ]:
feh = results_dict['goals_grouped_by_document'][['Goal','Count']].to_dict('index')
print(feh)

In [ ]:
for name in results_dict['goals_grouped_by_document']:
    print(name)
